In [1]:
# %pylab
%matplotlib inline
# %pylab ipympl
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import matplotlib.colors as colors
import scipy.integrate
import sys
import os
import time
import logging
import collections

import pickle
import matplotlib.patches as patches

# my modules
# sys.path.insert(0,'/user/scratch14_csa/dstephens/OldNugridpy/')
import nugridpy.ascii_table as table
from nugridpy import nugridse as nuse
from astropy import constants as const
from astropy import units as u
import nugridpy.utils as utils
import nugridpy.astronomy as ast

Msun = (const.M_sun.to(u.g)).value
Rsun = (const.R_sun.to(u.cm)).value

sys.path.insert(0,'/user/scratch14_csa/dstephens/advection-streams/')
import hydromixing.mixer as mixer

sys.path.insert(0,'/user/scratch14_csa/dstephens/PyPPM/')
from ppmpy import ppm

import matplotlib as mpl

cb = utils.linestylecb # colours

# plot things
ifig = 0
celln = 0
ptrack = {}

# turn off matplotlib messages
logging.getLogger("matplotlib").setLevel(logging.ERROR)

plt.rc('xtick', labelsize=7)
plt.rc('ytick', labelsize=7)

In [2]:
def get_nsubt(initbase, simDump0, simDumpMax):
    
    # return an array with the nsubt timesteps taken at every dump within the simulation time
    
    # what dumps are we going to read in
    dumps = list(range(simDump0, simDumpMax+1))
    nsubt = np.zeros(len(dumps), dtype=np.int16)
    stream = mixer.Stream(initbase, 'advection', initdump=simDump0)
    stream.readFile(initbase, 'advection', 'shell', simDump0)
    
    for i,dump in enumerate(dumps):

        # read in file
        stream.readFile(initbase, 'advection', 'shell', dump)
        nsubt[i] = int(stream.shellHeaders['subtimesteps'])
        
    return nsubt

def removelists(se, get_quantity, cycle):
    
    # with all mppnp runs there is a possibility of getting a list of results. Take the last one
    got_quantity = se.get(int(cycle), get_quantity)
    
    # I have ~250 zones in all, if its len is small, it is a list
    if len(got_quantity) < 100:
        got_quantity = got_quantity[0]
    
    return got_quantity

# get the correpsonding cycle from a ppmstar dump
def dump2cycle(nrepeat, nsubt, dump, simDump0):
    if dump == simDump0:
        return int(nrepeat)
    else:
        return int(nrepeat + nsubt[0:(dump-simDump0)].sum())

# use remove print to console at certain points then return
class RedirectStdStreams(object):
    def __init__(self, stdout=None, stderr=None):
        self._stdout = stdout or sys.stdout
        self._stderr = stderr or sys.stderr

    def __enter__(self):
        self.old_stdout, self.old_stderr = sys.stdout, sys.stderr
        self.old_stdout.flush(); self.old_stderr.flush()
        sys.stdout, sys.stderr = self._stdout, self._stderr

    def __exit__(self, exc_type, exc_value, traceback):
        self._stdout.flush(); self._stderr.flush()
        sys.stdout = self.old_stdout
        sys.stderr = self.old_stderr

In [3]:
# which ppmstar run you are comparing profiles with?
myrun = 'n16'

information that is always needed

In [4]:
# ppmstar rprof to get
all_runs = ['n15','n16','n17']
all_rprofdirs = ['/data/ASDR/PPMstar/LowZRAWD/N15-LowZRAWD-768-10x-burn-moms/prfs/',
                '/data/ASDR/PPMstar/LowZRAWD/N16-LowZRAWD-1536-10x-burn-moms/prfs/',
                '/data/ASDR/PPMstar/LowZRAWD/N17-LowZRAWD-1152-100x-burn-moms/prfs/']

allrprofs = dict(zip(all_runs, all_rprofdirs))

# where are the runs shell files? We also need each offset of repeating cycles and their nsubt
initbase_list = ['/data/niagara_project/projects/3D1D-advection/N15_shell_files/N15-standard-mppnp/N15-standard-mppnp',
                '/data/niagara_project/projects/3D1D-advection/N16_shell_files/N16-standard-mppnp/N16-standard-mppnp',
                '/data/niagara_project/projects/3D1D-advection/N17_shell_files/N17-standard-mppnp/N17-standard-mppnp']
initbase = dict(zip(all_runs, initbase_list))

mysimDump0 = 100
allFiles = [afile for afile in os.listdir(os.path.dirname(initbase[myrun]))]
allShellFiles = [afile for afile in allFiles if afile.endswith('.shell')]
mysimDumpMax = len(allShellFiles) + mysimDump0 - 1

# the number of nsubt for a given run by counting the subtime steps
run_nsubt = get_nsubt(initbase[myrun], mysimDump0, mysimDumpMax)

Collect the data for plotting. Includes plotting lists for the data

In [5]:
# my rprof from PPMStar
myrprof = ppm.RprofSet(allrprofs[myrun])

1618 rprof files found in '/data/ASDR/PPMstar/LowZRAWD/N16-LowZRAWD-1536-10x-burn-moms/prfs/.
Dump numbers range from 0 to 1617.


In [6]:
# rprof constants
atomicnoH = 1.
atomicnoC12 = 12.
atomicnocld = myrprof.get('atomicnocld',fname=0)
atomicnoair = myrprof.get('atomicnoair',fname=0)
fkcld = myrprof.get('fkcld',fname=0)
fkair = myrprof.get('fkair',fname=0)
cldmu = myrprof.get('cldmu',fname=0)
airmu = myrprof.get('airmu',fname=0)

In [7]:
tau_conv = 16

Change what you want to plot with **species_label**, change the dump with **dump_plot**. You can remove Jones easily from the plotting with removing 'Jones' from the list of **se_labels**

In [8]:
# labels for when we loop through runs
se_labels = ['up', 'down', 'DFV', 'Jones']
se_linestyle = ['--', '-.', '-', ':']

# labels for when we loop through species
species_label = ['Kr-88', 'Kr-89', 'Kr-90']

species_color = list(map(lambda x: cb(x+1)[2], np.arange(len(species_label))))

# dump to plot
dump_plot = 1200

# plot ylimit
ylims = [1e-20, 1e-8]

# savefig?
ifsavefig = False
savefig = 'N16-Kr-isotopes.png'

What are the the number of repeats of the first cycle? What are the number of subtimesteps between each dump?

In [9]:
se_nsubt = [run_nsubt, run_nsubt, run_nsubt, run_nsubt]
se_nrepeat = [100, 100, 100, 100]

Based on the plotting dump, we will read in the appropriate se file to speed things up

In [10]:
ls -d /data/niagara_project/projects/3D1D-advection/mppnp-paper-data/*mp11

ls: cannot access '/data/niagara_project/projects/3D1D-advection/mppnp-paper-data/*mp11': No such file or directory


In [11]:
# mppnp data
# mppnp1 = '/user/cedar_scratch_djstephe/mppnp/H5_out_up_ppm_N16_ext_nsubt1_new'
# mppnp2 = '/user/cedar_scratch_djstephe/mppnp/H5_out_down_ppm_N16_ext_nsubt1_new'
# mppnp3 = '/user/niagara_scratch_pavelden/RUN_MP5/H5_out'
# mppnp4 = '/user/cedar_scratch_djstephe/mppnp/H5_out_mes_N16_ext_nsubt1_new'

mppnp1 = '/data/niagara_project/projects/3D1D-advection/mppnp-paper-data/H5_out_up_mp6'
mppnp2 = '/data/niagara_project/projects/3D1D-advection/mppnp-paper-data/H5_out_down_mp6'
mppnp3 = '/data/niagara_project/projects/3D1D-advection/mppnp-paper-data/H5_out_mes_mp5'
mppnp4 = '/data/niagara_project/projects/3D1D-advection/mppnp-paper-data/H5_out_mes_mp11'
mppnp_dirs = [mppnp1, mppnp2, mppnp3, mppnp4]

# to work with the mppnp data
se_blocks = [1000, 1000, 1000, 10000]
se_suffix = ['out_up.h5', 'out_down.h5', 'out.h5', 'out.h5']
se_name = 'my_test_adv'

# determine the "block" we want to read
block_name = []
for i,block in enumerate(se_blocks):
    cycle = dump2cycle(se_nrepeat[i], se_nsubt[i], dump_plot, mysimDump0)
    block_name.append(cycle - cycle%block+1)
    
se_files = []
devnull = open(os.devnull, 'w')
with RedirectStdStreams(stdout=devnull, stderr=devnull):
    
    for mppnp_dir, block in zip(mppnp_dirs, block_name):
        se_files.append(nuse.se(mppnp_dir, pattern=str(block)))

    myfacrad = (se_files[0].get('radius_unit') / Rsun)

FileNotFoundError: [Errno 2] No such file or directory: '/data/niagara_project/projects/3D1D-advection/mppnp-paper-data/H5_out_mes_mp11'

## Plot

In [ ]:
# celln
celln += 1

# configure plots
close_local = ppm.close_plot(celln,ifig,ptrack)
if close_local[0]:
    plt.close(fig); ifig += 1; fig = plt.figure(ifig,figsize=(4,3),dpi=300)
    ppm.add_plot(celln,ifig,ptrack)
else:
    ifig += 1; fig = plt.figure(ifig,figsize=(4,3),dpi=300)
    ppm.add_plot(celln,ifig,ptrack)

ax = fig.add_subplot(111)
cbc = 0

# get the radius
r = removelists(se_files[0], 'radius', dump2cycle(se_nrepeat[0], se_nsubt[0], dump_plot, mysimDump0)) / myfacrad

# PLOT PPMSTAR FIRST
# r_rprof = myrprof.get('R',fname=dump_plot,resolution='l')
# Xcld_rprof = myrprof.compute_Xcld(dump_plot)
# XH_rprof = Xcld_rprof * fkcld * atomicnoH / atomicnocld

# ax.plot(r_rprof, XH_rprof, color=cb(cbc)[2], ls=cb(cbc)[0], label='PPMStar')
# cbc += 1

# now loop over the se_files
cb_lines = -1

for i,myse in enumerate(se_files):
    
    # change the color of the lines based on species while linestyle is run based
    cb_lines += 1
    cb_color = 0
    
    for j,specie in enumerate(species_label):
        
        # get the species
        aspecies = removelists(myse, specie, dump2cycle(se_nrepeat[i], se_nsubt[i], dump_plot, mysimDump0))
        
        # if neutron is aspecies
#         aspecies *= removelists(myse, 'rho', dump2cycle(se_nrepeat[i], se_nsubt[i], dump_plot, mysimDump0)) / 1.66e-24
        
        if i == 0:
            ax.plot(r, aspecies, color=species_color[cb_color], ls=se_linestyle[cb_lines], 
                    label=specie)
        else:
            ax.plot(r, aspecies, color=species_color[cb_color], ls=se_linestyle[cb_lines])
        cb_color += 1

# other plot details
ax.set_xlabel('r / Mm')
ax.set_ylabel('X')
ax.set_yscale('log')
ax.set_ylim(ylims)
ax.set_xlim([r.min(), r.max()])
ax.legend(frameon=False, loc='lower right', fontsize=7)

# savefig
fig.tight_layout()

if ifsavefig:
    fig.savefig(savefig)